In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 16384 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "codellama/CodeLlama-13b-Instruct-hf",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


...


Loading checkpoint shards:  33%|████████████████████▎                                        | 1/3 [00:31<01:03, 31.79s/it]


Loading checkpoint shards:  67%|████████████████████████████████████████▋                    | 2/3 [00:53<00:25, 25.59s/it]


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████| 3/3 [01:04<00:00, 19.28s/it]


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████| 3/3 [01:04<00:00, 21.60s/it]

codellama/CodeLlama-13b-Instruct-hf does not have a padding token! Will use pad_token = <unk>.


In [ ]:
from datasets import load_dataset

# Define the paths to your dataset files
data_files = {
    'train': 'vuln_data_train.csv',
    'val': 'vuln_data_val.csv',
    'test': 'vuln_data_test.csv'
}

# Load the dataset
dataset = load_dataset('csv', data_files=data_files)

import random
from datasets import concatenate_datasets, DatasetDict

random.seed(42)

SYSTEM_PROMPT = [
    """You are the smartest AI solidity smart contract security auditor in the world""",
]

PROMPTS = [
    """The given Solidity function is vulnerable and there was an explanation about the vulnerability. Give your patch recommendation based on that vulnerability. Just straight to the recommendation without any additional information.
    
    This is the vulnerable functions:
    ```solidity
    {code}
    ```
    
This is the vulnerability explanation: 
{explanation}

Recommendation:
""",
]

def make_conversational(examples):
    code = examples['vuln_code']
    desc = examples['vuln_explanation']
    recom = examples['vuln_recommendation']
    selected_system_prompt = random.choice(SYSTEM_PROMPT)
    selected_user_prompt = random.choice(PROMPTS) 
    conversation = [{"role": "system", "content": selected_system_prompt}, 
                    {"role": "user", "content": selected_user_prompt.format(code=code, explanation=desc)}]
    return { "conversations" : conversation, "label": recom}

dataset = dataset.map(make_conversational)

test_dataset = dataset['test']

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True,)


Map:   0%|                                                                                  | 0/662 [00:00<?, ? examples/s]


Map: 100%|██████████████████████████████████████████████████████████████████████| 662/662 [00:00<00:00, 5370.19 examples/s]


Map: 100%|██████████████████████████████████████████████████████████████████████| 662/662 [00:00<00:00, 5234.03 examples/s]


Map:   0%|                                                                                  | 0/142 [00:00<?, ? examples/s]


Map: 100%|██████████████████████████████████████████████████████████████████████| 142/142 [00:00<00:00, 4813.64 examples/s]

...


Map: 100%|██████████████████████████████████████████████████████████████████████| 662/662 [00:00<00:00, 7682.39 examples/s]


Map:   0%|                                                                                  | 0/142 [00:00<?, ? examples/s]


Map: 100%|██████████████████████████████████████████████████████████████████████| 142/142 [00:00<00:00, 5496.82 examples/s]


Map:   0%|                                                                                  | 0/143 [00:00<?, ? examples/s]


Map: 100%|██████████████████████████████████████████████████████████████████████| 143/143 [00:00<00:00, 6817.91 examples/s]

In [ ]:
import re
from unsloth.chat_templates import get_chat_template
from tqdm import tqdm  # Import tqdm for progress bar
import pandas as pd

def extract_llm_res(raw_llm):
    # Regex pattern to extract the assistant's response after the "Vulnerability Explanation" section
    pattern = r'\[/INST](.*?)</s>'

    match = re.search(pattern, raw_llm, re.DOTALL)

    if match:
        assistant_response = match.group(1).strip()
        return assistant_response
    else:
        print(f"No response found. Response: {raw_llm}")
        return raw_llm

# Set up the tokenizer and model for fast inference
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama",
)
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

# Initialize the result collection
results = []

# Iterate over the dataset with tqdm for progress bar
for idx, example in tqdm(enumerate(test_dataset), total=len(test_dataset), desc="Processing dataset"):
    messages = example['conversations']  # Get the conversation for the current entry
    
    # Tokenize input
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,  # Must add for generation
        return_tensors="pt",
    ).to("cuda")

    # Get LLM output
    outputs = model.generate(input_ids=inputs, max_new_tokens=1024, use_cache=True, repetition_penalty=1.2,
                             temperature=0.1, min_p=0.1)
    raw_out = tokenizer.batch_decode(outputs)[0]
    
    # Extract LLM response
    llm_output = extract_llm_res(raw_out)
    
    # Get the ground truth for the current entry
    ground_truth = example['label']
    
    results.append({
        "vuln_code": example['vuln_code'],
        "ground_truth": ground_truth,
        "llm_output": llm_output
    })

# After the loop, you can convert the results list to a DataFrame or another suitable format if needed
verification_results_df = pd.DataFrame(results)


Processing dataset:   0%|                                                                          | 0/143 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Processing dataset:   1%|▍                                                                 | 1/143 [00:14<33:53, 14.32s/it]


Processing dataset:   1%|▉                                                                 | 2/143 [00:20<22:45,  9.68s/it]


Processing dataset:   2%|█▍                                                                | 3/143 [00:27<19:22,  8.30s/it]

...


Processing dataset:  98%|██████████████████████████████████████████████████████████████▋ | 140/143 [37:20<01:15, 25.21s/it]


Processing dataset:  99%|███████████████████████████████████████████████████████████████ | 141/143 [37:39<00:47, 23.51s/it]


Processing dataset:  99%|███████████████████████████████████████████████████████████████▌| 142/143 [37:52<00:20, 20.14s/it]


Processing dataset: 100%|████████████████████████████████████████████████████████████████| 143/143 [38:02<00:00, 17.30s/it]


Processing dataset: 100%|████████████████████████████████████████████████████████████████| 143/143 [38:02<00:00, 15.96s/it]

In [ ]:
verification_results_df.to_csv("zeroshot_codellama13b_recommendation.csv", index=False)